IMPORT

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from torchtext.vocab import vocab
from zipfile import ZipFile
from urllib.request import urlretrieve
import os
import time
import math
from tempfile import TemporaryDirectory

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [ ]:

# Downloading dataset
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-1m.zip", "movielens.zip")
ZipFile("movielens.zip", "r").extractall()

# Loading dataset
users = pd.read_csv(
    "ml-1m/users.dat",
    sep="::",
    names=["user_id", "sex", "age_group", "occupation", "zip_code"],
    engine='python'
)

ratings = pd.read_csv(
    "ml-1m/ratings.dat",
    sep="::",
    names=["user_id", "movie_id", "rating", "unix_timestamp"],
    engine='python'
)
movies = pd.read_csv(
    "ml-1m/movies.dat", sep="::", names=["movie_id", "title", "genres"], encoding='latin-1', engine='python'
)


#PREPROCESSING

In [ ]:
# Preventing ids to be written as integer or float data type
users["user_id"] = users["user_id"].apply(lambda x: f"user_{x}")
movies["movie_id"] = movies["movie_id"].apply(lambda x: f"movie_{x}")
ratings["movie_id"] = ratings["movie_id"].apply(lambda x: f"movie_{x}")
ratings["user_id"] = ratings["user_id"].apply(lambda x: f"user_{x}")
users["age_group"] = users["age_group"].apply(lambda x: f"age_group_{x}")
users["occupation"] = users["occupation"].apply(lambda x: f"occupation_{x}")

In [ ]:
# Generate a list of unique movie ids
movie_ids = movies.movie_id.unique()

# Counter is used to feed movies to movie_vocab
movie_counter = Counter(movie_ids)

# Generating vocabulary
movie_vocab = vocab(movie_counter, specials=['<unk>'])

# For indexing input ids
movie_vocab_stoi = movie_vocab.get_stoi()

# Movie to title mapping dictionary
movie_title_dict = dict(zip(movies.movie_id, movies.title))

# Similarly generating a vocabulary for user ids
user_ids = users.user_id.unique()
user_counter = Counter(user_ids)
user_vocab = vocab(user_counter, specials=['<unk>'])
user_vocab_stoi = user_vocab.get_stoi()

In [ ]:
age_groups = users.age_group.unique()
age_counter = Counter(age_groups)
age_vocab = vocab(age_counter, specials=['<unk>'])
age_vocab_stoi = age_vocab.get_stoi()

In [ ]:
occ_groups = users.occupation.unique()
occ_counter = Counter(occ_groups)
occ_vocab = vocab(occ_counter, specials=['<unk>'])
occupation_stoi = occ_vocab.get_stoi()

In [ ]:
# Generating vocabularies for additional user features
sex_vocab = vocab(Counter(users.sex), specials=['<unk>'])
sex_stoi = sex_vocab.get_stoi()

In [ ]:
movie_genre = dict(zip(movies.movie_id, movies.genres))
movie_genre = {key: value.replace('|', ', ') for key, value in movie_genre.items()}

In [ ]:
movie_rating = dict(zip(movies.movie_id, ratings.rating))

In [ ]:
# Group ratings by user_id in order of increasing unix_timestamp.
ratings_group = ratings.sort_values(by=["unix_timestamp"]).groupby("user_id")

ratings_data = pd.DataFrame(
    data={
        "user_id": list(ratings_group.groups.keys()),
        "movie_ids": list(ratings_group.movie_id.apply(list)),
        "timestamps": list(ratings_group.unix_timestamp.apply(list)),
    }
)

In [ ]:
# Sequence length, min history count and window slide size
sequence_length = 4
min_history = 1
step_size = 2

# Creating sequences from lists with sliding window
def create_sequences(values, window_size, step_size, min_history):
    sequences = []
    start_index = 0
    while len(values[start_index:]) > min_history:
        seq = values[start_index : start_index + window_size]
        sequences.append(seq)
        start_index += step_size
    return sequences

ratings_data.movie_ids = ratings_data.movie_ids.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size, min_history)
)

del ratings_data["timestamps"]

# Sub-sequences are exploded.
# Since there might be more than one sequence for each user.
ratings_data_transformed = ratings_data[["user_id", "movie_ids"]].explode(
    "movie_ids", ignore_index=True
)

ratings_data_transformed.rename(
    columns={"movie_ids": "sequence_movie_ids"},
    inplace=True,
)

In [ ]:
# Adding user features to the dataset
user_features = users[["user_id", "sex", "age_group", "occupation"]]
ratings_data_transformed = ratings_data_transformed.merge(user_features, on="user_id")

# Random indexing
random_selection = np.random.rand(len(ratings_data_transformed.index)) <= 0.85

# Split train data
df_train_data = ratings_data_transformed[random_selection]
train_data_raw = df_train_data[["user_id", "sequence_movie_ids", "sex", "age_group", "occupation"]].values

# Split test data
df_test_data = ratings_data_transformed[~random_selection]
test_data_raw = df_test_data[["user_id", "sequence_movie_ids", "sex", "age_group", "occupation"]].values

In [ ]:
# PyTorch Dataset for user interactions
class MovieSeqDataset(Dataset):
    # Initialize dataset
    def __init__(self, data, movie_vocab_stoi, user_vocab_stoi, sex_stoi, age_group_stoi, occupation_stoi):
        self.data = data
        self.movie_vocab_stoi = movie_vocab_stoi
        self.user_vocab_stoi = user_vocab_stoi
        self.sex_stoi = sex_stoi
        self.age_group_stoi = age_group_stoi
        self.occupation_stoi = occupation_stoi

    def __len__(self):
        return len(self.data)

    # Fetch data from the dataset
    def __getitem__(self, idx):
        user, movie_sequence, sex, age_group, occupation = self.data[idx]
        movie_data = [self.movie_vocab_stoi[item] for item in movie_sequence]
        user_data = self.user_vocab_stoi[user]
        sex_data = self.sex_stoi[sex]
        age_group_data = self.age_group_stoi[age_group]
        occupation_data = self.occupation_stoi[occupation]
        return torch.tensor(movie_data), torch.tensor(user_data), torch.tensor(sex_data), torch.tensor(age_group_data), torch.tensor(occupation_data)

In [ ]:
# Collate function and padding
def collate_batch(batch):
    movie_list = [item[0] for item in batch]
    user_list = [item[1] for item in batch]
    sex_list = [item[2] for item in batch]
    age_group_list = [item[3] for item in batch]
    occupation_list = [item[4] for item in batch]
    return pad_sequence(movie_list, padding_value=movie_vocab_stoi['<unk>'], batch_first=True), torch.stack(user_list), torch.stack(sex_list), torch.stack(age_group_list), torch.stack(occupation_list)

BATCH_SIZE = 256
# Create instances of your Dataset for each set
train_dataset = MovieSeqDataset(train_data_raw, movie_vocab_stoi, user_vocab_stoi, sex_stoi, age_vocab_stoi, occupation_stoi)
val_dataset = MovieSeqDataset(test_data_raw, movie_vocab_stoi, user_vocab_stoi, sex_stoi, age_vocab_stoi, occupation_stoi)
# Create DataLoaders
train_iter = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
val_iter = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

#DEF - MODEL

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)

        # div_term is used in the calculation of the sinusoidal values.
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))

        # Initializing positional encoding matrix with zeros.
        pe = torch.zeros(max_len, 1, d_model)

        # Calculating the positional encodings.
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape `[seq_len, batch_size, embedding_dim]`
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, ntoken: int, nuser: int, nsex: int, nage_group: int, noccupation: int, d_model: int, nhead: int, d_hid: int, nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        # Positional encoder
        self.pos_encoder = PositionalEncoding(d_model, dropout)

        # Multihead attention mechanism
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)

        # Embedding layers
        self.movie_embedding = nn.Embedding(ntoken, d_model)
        self.user_embedding = nn.Embedding(nuser, d_model)
        self.sex_embedding = nn.Embedding(nsex, d_model)
        self.age_group_embedding = nn.Embedding(nage_group, d_model)
        self.occupation_embedding = nn.Embedding(noccupation, d_model)

        # Defining the size of the input to the model
        self.d_model = d_model

        # Linear layer to map the output to movie vocabulary
        self.linear = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        # Initializing the weights of the embedding and linear layers
        initrange = 0.1
        self.movie_embedding.weight.data.uniform_(-initrange, initrange)
        self.user_embedding.weight.data.uniform_(-initrange, initrange)
        self.sex_embedding.weight.data.uniform_(-initrange, initrange)
        self.age_group_embedding.weight.data.uniform_(-initrange, initrange)
        self.occupation_embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, user: Tensor, sex: Tensor, age_group: Tensor, occupation: Tensor, src_mask: Tensor = None) -> Tensor:
        # Embedding movie ids and user features
        movie_embed = self.movie_embedding(src) * math.sqrt(self.d_model)
        user_embed = self.user_embedding(user) * math.sqrt(self.d_model)
        sex_embed = self.sex_embedding(sex) * math.sqrt(self.d_model)
        age_group_embed = self.age_group_embedding(age_group) * math.sqrt(self.d_model)
        occupation_embed = self.occupation_embedding(occupation) * math.sqrt(self.d_model)

        # Concatenate user feature embeddings with movie embeddings
        combined_embed = movie_embed + user_embed.unsqueeze(1) + sex_embed.unsqueeze(1) + age_group_embed.unsqueeze(1) + occupation_embed.unsqueeze(1)

        # Apply positional encoding
        combined_embed = self.pos_encoder(combined_embed)

        # Generate output with final layers
        output = self.transformer_encoder(combined_embed, src_mask)

        # Apply the linear layer
        output = self.linear(output)
        return output

In [ ]:
ntokens = len(movie_vocab)  # size of vocabulary
nusers = len(user_vocab)
nsex = len(sex_vocab)
nage_group = len(age_vocab)
noccupation = len(occ_vocab)
emsize = 128  # embedding dimension
d_hid = 128  # dimension of the feedforward network model
nlayers = 2  # number of `nn.TransformerEncoderLayer`
nhead = 2  # number of heads in `nn.MultiheadAttention`
dropout = 0.2  # dropout probability

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TransformerModel(ntokens, nusers, nsex, nage_group, noccupation, emsize, nhead, d_hid, nlayers, dropout).to(device)

criterion = nn.CrossEntropyLoss()
lr = 1.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


# TRAIN AND EVALUATION

In [ ]:
def train(model: nn.Module, train_iter, epoch) -> None:
    # Switch to training mode
    model.train()
    total_loss = 0.
    log_interval = 200
    start_time = time.time()

    for i, (movie_data, user_data, sex_data, age_group_data, occupation_data) in enumerate(train_iter):
        # Load movie sequence and user features
        movie_data, user_data, sex_data, age_group_data, occupation_data = movie_data.to(device), user_data.to(device), sex_data.to(device), age_group_data.to(device), occupation_data.to(device)

        # Split movie sequence to inputs and targets
        inputs, targets = movie_data[:, :-1], movie_data[:, 1:]
        targets_flat = targets.reshape(-1)

        # Predict movies
        output = model(inputs, user_data, sex_data, age_group_data, occupation_data)
        output_flat = output.reshape(-1, ntokens)

        # Backpropogation process
        loss = criterion(output_flat, targets_flat)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        # Results
        if i % log_interval == 0 and i > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

In [ ]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    # Switch the model to evaluation mode.
    # This is necessary for layers like dropout,
    model.eval()
    total_loss = 0.

    with torch.no_grad():
        for i, (movie_data, user_data, sex_data, age_group_data, occupation_data) in enumerate(eval_data):
            # Load movie sequence and user features
            movie_data, user_data, sex_data, age_group_data, occupation_data = movie_data.to(device), user_data.to(device), sex_data.to(device), age_group_data.to(device), occupation_data.to(device)
            # Split movie sequence to inputs and targets
            inputs, targets = movie_data[:, :-1], movie_data[:, 1:]
            targets_flat = targets.reshape(-1)
            # Predict movies
            output = model(inputs, user_data, sex_data, age_group_data, occupation_data)
            output_flat = output.reshape(-1, ntokens)
            # Calculate loss
            loss = criterion(output_flat, targets_flat)
            total_loss += loss.item()
    return total_loss / (len(eval_data) - 1)

In [ ]:
best_val_loss = float('inf')
epochs = 10

In [ ]:
with TemporaryDirectory() as tempdir:
    best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()

        # Training
        train(model, train_iter, epoch)

        # Evaluation
        val_loss = evaluate(model, val_iter)

        # Compute the perplexity of the validation loss
        val_ppl = math.exp(val_loss)
        elapsed = time.time() - epoch_start_time

        # Results
        print('-' * 89)
        print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
            f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
        print('-' * 89)

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_params_path)

        scheduler.step()
    model.load_state_dict(torch.load(best_model_params_path)) # load best model states

| epoch   1 lr 1.00 | ms/batch 311.64 | loss  7.85 | ppl  2554.17
| epoch   1 lr 1.00 | ms/batch 315.46 | loss  7.64 | ppl  2070.64
| epoch   1 lr 1.00 | ms/batch 304.22 | loss  7.59 | ppl  1987.07
| epoch   1 lr 1.00 | ms/batch 310.49 | loss  7.56 | ppl  1924.44
| epoch   1 lr 1.00 | ms/batch 305.56 | loss  7.54 | ppl  1876.36
| epoch   1 lr 1.00 | ms/batch 310.51 | loss  7.53 | ppl  1855.03
| epoch   1 lr 1.00 | ms/batch 302.92 | loss  7.49 | ppl  1794.57
| epoch   1 lr 1.00 | ms/batch 310.39 | loss  7.42 | ppl  1669.57
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 542.28s | valid loss  7.35 | valid ppl  1550.19
-----------------------------------------------------------------------------------------
| epoch   2 lr 0.95 | ms/batch 312.38 | loss  7.24 | ppl  1391.23
| epoch   2 lr 0.95 | ms/batch 309.52 | loss  7.12 | ppl  1233.06
| epoch   2 lr 0.95 | ms/batch 304.75 | loss  7.01 | ppl  1104.93
| epoch   2 lr 0.95

In [ ]:
# Movie id decoder
movie_vocab_itos = movie_vocab.get_itos()

# A placeholders to store results of recommendations
transformer_reco_results = list()

# Get top 10 movies
k = 10
# Iterate over the validation data
for i, (movie_data, user_data, sex_data, age_group_data, occupation_data) in enumerate(val_iter):
    # Feed the input and get the outputs
    movie_data, user_data, sex_data, age_group_data, occupation_data = movie_data.to(device), user_data.to(device), sex_data.to(device), age_group_data.to(device), occupation_data.to(device)
    inputs, targets = movie_data[:, :-1], movie_data[:, 1:]
    output = model(inputs, user_data, sex_data, age_group_data, occupation_data)
    output_flat = output.reshape(-1, ntokens)
    targets_flat = targets.reshape(-1)

    # Reshape the output_flat to get top predictions
    outputs = output_flat.reshape(output_flat.shape[0] // inputs.shape[1],
                                  inputs.shape[1],
                                  output_flat.shape[1])[:, -1, :]
    # k + len(inputs) = 13 movies obtained
    # In order to prevent to recommend already watched movies
    values, indices = outputs.topk(k + inputs.shape[1], dim=-1)

    for sub_sequence, sub_indice_org in zip(movie_data, indices):
        sub_indice_org = sub_indice_org.cpu().detach().numpy()
        sub_sequence = sub_sequence.cpu().detach().numpy()

        # Generate mask array to eliminate already watched movies
        mask = np.isin(sub_indice_org, sub_sequence[:-1], invert=True)

        # After masking get top k movies
        sub_indice = sub_indice_org[mask][:k]

        # Generate results array
        transformer_reco_result = np.isin(sub_indice, sub_sequence[-1]).astype(int)

        # Decode movie to search in popular movies
        target_movie_decoded = movie_vocab_itos[sub_sequence[-1]]

        transformer_reco_results.append(transformer_reco_result)

# OUTPUT

In [ ]:
def generate_recommendation(user_id, movie_sequence, sex, age_group, occupation, k=10):
    model.eval()
    input_sequence = movie_sequence[:-1]
    user_tensor = torch.tensor(user_vocab_stoi[user_id]).unsqueeze(0).to(device)
    sex_tensor = torch.tensor(sex_stoi[sex]).unsqueeze(0).to(device)
    age_group_tensor = torch.tensor(age_vocab_stoi[age_group]).unsqueeze(0).to(device)
    occupation_tensor = torch.tensor(occupation_stoi[occupation]).unsqueeze(0).to(device)
    movie_tensor = torch.tensor([[movie_vocab_stoi[movie_id]] for movie_id in input_sequence]).to(device)

    with torch.no_grad():
        predictions = model(movie_tensor, user_tensor, sex_tensor, age_group_tensor, occupation_tensor)

    # The output is a probability distribution over the next movie.
    # Topk to get most probable movies
    values, indices = predictions.topk(k + len(input_sequence), dim=-1)
    # Eliminate already watched movies
    indices = [indice for indice in indices[-1, :][0] if indice not in movie_tensor][len(input_sequence):]

    predicted_movies = [movie_title_dict[movie_vocab_itos[movie]] for movie in indices]
    predicted_genres = [movie_genre[movie_vocab_itos[movie]] for movie in indices]
    predicted_ratings = [movie_rating[movie_vocab_itos[movie]] for movie in indices]

    return predicted_movies, predicted_genres, predicted_ratings

In [ ]:
occupation_mapping = {
    "occupation_0": "other or not specified",
    "occupation_1": "academic/educator",
    "occupation_2": "artist",
    "occupation_3": "clerical/admin",
    "occupation_4": "college/grad student",
    "occupation_5": "customer service",
    "occupation_6": "doctor/health care",
    "occupation_7": "executive/managerial",
    "occupation_8": "farmer",
    "occupation_9": "homemaker",
    "occupation_10": "K-12 student",
    "occupation_11": "lawyer",
    "occupation_12": "programmer",
    "occupation_13": "retired",
    "occupation_14": "sales/marketing",
    "occupation_15": "scientist",
    "occupation_16": "self-employed",
    "occupation_17": "technician/engineer",
    "occupation_18": "tradesman/craftsman",
    "occupation_19": "unemployed",
    "occupation_20": "writer"
}

In [ ]:
age_group_mapping = {
    "age_group_1": "Under 18",
    "age_group_18": "18-24",
    "age_group_25": "25-34",
    "age_group_35": "35-44",
    "age_group_45": "45-49",
    "age_group_50": "50-55",
    "age_group_56": "56+"
}

In [ ]:
user_sex = dict(zip(users.user_id, users.sex))
user_age = dict(zip(users.user_id, users.age_group))
user_occ = dict(zip(users.user_id, users.occupation))

#MODEL JADI

In [ ]:
def proses_model():
    userid = input("Masukkan id user: ")
    movie1 = input("Masukkan id film 1: ")
    movie2 = input("Masukkan id film 2: ")
    movie3 = input("Masukkan id film 3: ")

    userid = "user_" + userid

    movie1 = "movie_" + movie1
    movie2 = "movie_" + movie2
    movie3 = "movie_" + movie3

    movie = [movie1, movie2, movie3]

    moviez = [movie_title_dict[ea_movie] for ea_movie in movie]
    genres = [movie_genre[ea_movie] for ea_movie in movie]
    rates = [movie_rating[ea_movie] for ea_movie in movie]
    usersex = user_sex[userid]
    userage = user_age[userid]
    userocc = user_occ[userid]

    recom = generate_recommendation(userid, movie, usersex, userage, userocc)
    judul, genrez, ratingz = recom

    print(" ")
    print(f"User: {userid} | Sex: {usersex} | Age Group: {age_group_mapping.get(userage)} | Occupation: {occupation_mapping.get(userocc)}")
    print(" ")

    print("Film input:")
    for i in range(3):
        hasil1 = [f"- {moviez[i]} | Genre: {genres[i]} | Rating: {rates[i]}"]
        for item in hasil1:
          print(item)

    print(" ")

    print("Film Rekomendasi:")
    for i in range(11):
        try:
            hasil2 = [f"- {judul[i]} | Genre: {genrez[i]} | Rating: {ratingz[i]}"]
            for item in hasil2:
                print(item)
        except IndexError:
            print(" ")

    print(" ")
    print(" ")

In [ ]:
def cek_history():
    userid = input("Masukkan id user: ")
    userid = "user_" + userid

    coba = ratings.sort_values(by=["unix_timestamp"]).groupby("user_id")
    coba = pd.DataFrame(
      data={
        "user_id": list(ratings_group.groups.keys()),
        "movie_ids": list(ratings_group.movie_id.apply(list)),
      }
    )

    histori = dict(zip(coba.user_id, coba.movie_ids))
    hiss = histori[userid]

    movie_his = [movie_title_dict[ea_movie] for ea_movie in hiss]
    genres = [movie_genre[ea_movie] for ea_movie in hiss]
    rates = [movie_rating[ea_movie] for ea_movie in hiss]

    usersex = user_sex[userid]
    userage = user_age[userid]
    userocc = user_occ[userid]

    print(" ")
    print(f"User: {userid} | Sex: {usersex} | Age Group: {age_group_mapping.get(userage)} | Occupation: {occupation_mapping.get(userocc)}")
    print(" ")

    print("History Film User:")
    for i in range(len(movie_his)):
        try:
            hasil2 = [f"- {movie_his[i]} | Genre: {genres[i]} | Rating: {rates[i]}"]
            for item in hasil2:
                print(item)
        except IndexError:
            print(" ")

    print(" ")
    print(" ")

# CEK HASIL

##GENRE CHILDREN + ANIMATION

In [ ]:
proses_model()

Masukkan id user: 1
Masukkan id film 1: 1566
Masukkan id film 2: 364
Masukkan id film 3: 1064
 
User: user_1 | Sex: F | Age Group: Under 18 | Occupation: K-12 student
 
Film input:
- Hercules (1997) | Genre: Adventure, Animation, Children's, Comedy, Musical | Rating: 2
- Lion King, The (1994) | Genre: Animation, Children's, Musical | Rating: 3
- Aladdin and the King of Thieves (1996) | Genre: Animation, Children's, Comedy | Rating: 5
 
Film Rekomendasi:
- James and the Giant Peach (1996) | Genre: Animation, Children's, Musical | Rating: 4
- Bug's Life, A (1998) | Genre: Animation, Children's, Comedy | Rating: 4
- Tarzan (1999) | Genre: Animation, Children's | Rating: 3
- Antz (1998) | Genre: Animation, Children's | Rating: 4
- Aladdin (1992) | Genre: Animation, Children's, Comedy, Musical | Rating: 3
- Hunchback of Notre Dame, The (1996) | Genre: Animation, Children's, Musical | Rating: 4
- Mary Poppins (1964) | Genre: Children's, Comedy, Musical | Rating: 2
- Mulan (1998) | Genre: Ani

In [ ]:
#cek histori 1
cek_history()

Masukkan id user: 1
 
User: user_1 | Sex: F | Age Group: Under 18 | Occupation: K-12 student
 
History Film User:
- Girl, Interrupted (1999) | Genre: Drama | Rating: 3
- Titanic (1997) | Genre: Drama, Romance | Rating: 4
- Back to the Future (1985) | Genre: Comedy, Sci-Fi | Rating: 4
- Cinderella (1950) | Genre: Animation, Children's, Musical | Rating: 5
- Meet Joe Black (1998) | Genre: Romance | Rating: 4
- Last Days of Disco, The (1998) | Genre: Drama | Rating: 4
- Erin Brockovich (2000) | Genre: Drama | Rating: 3
- To Kill a Mockingbird (1962) | Genre: Drama | Rating: 3
- Christmas Story, A (1983) | Genre: Comedy, Drama | Rating: 3
- Star Wars: Episode IV - A New Hope (1977) | Genre: Action, Adventure, Fantasy, Sci-Fi | Rating: 3
- Wallace & Gromit: The Best of Aardman Animation (1996) | Genre: Animation | Rating: 5
- One Flew Over the Cuckoo's Nest (1975) | Genre: Drama | Rating: 4
- Wizard of Oz, The (1939) | Genre: Adventure, Children's, Drama, Musical | Rating: 3
- Fargo (1996) 

##GENRE COMEDY

In [ ]:
proses_model()

Masukkan id user: 45
Masukkan id film 1: 19
Masukkan id film 2: 1439
Masukkan id film 3: 1444
 
User: user_45 | Sex: F | Age Group: 45-49 | Occupation: self-employed
 
Film input:
- Ace Ventura: When Nature Calls (1995) | Genre: Comedy | Rating: 5
- Meet Wally Sparks (1997) | Genre: Comedy | Rating: 3
- Guantanamera (1994) | Genre: Comedy | Rating: 2
 
Film Rekomendasi:
- Shakespeare in Love (1998) | Genre: Comedy, Romance | Rating: 4
- Sabrina (1995) | Genre: Comedy, Romance | Rating: 5
- Butcher's Wife, The (1991) | Genre: Comedy, Romance | Rating: 5
- Hope Floats (1998) | Genre: Comedy, Drama, Romance | Rating: 4
- Runaway Bride (1999) | Genre: Comedy, Romance | Rating: 2
- Mrs. Winterbourne (1996) | Genre: Comedy, Romance | Rating: 2
- I.Q. (1994) | Genre: Comedy, Romance | Rating: 1
- Nine Months (1995) | Genre: Comedy | Rating: 3
- Forget Paris (1995) | Genre: Comedy, Romance | Rating: 3
- Desperately Seeking Susan (1985) | Genre: Comedy, Romance | Rating: 4
 
 
 


In [ ]:
cek_history()

Masukkan id user: 45
 
User: user_45 | Sex: F | Age Group: 45-49 | Occupation: self-employed
 
History Film User:
- Sex, Lies, and Videotape (1989) | Genre: Drama | Rating: 5
- Gone with the Wind (1939) | Genre: Drama, Romance, War | Rating: 5
- Schindler's List (1993) | Genre: Drama, War | Rating: 4
- Tom Jones (1963) | Genre: Comedy | Rating: 3
- Fisher King, The (1991) | Genre: Comedy, Drama, Romance | Rating: 3
- Prizzi's Honor (1985) | Genre: Comedy, Drama, Romance | Rating: 4
- Annie Hall (1977) | Genre: Comedy, Romance | Rating: 3
- Goodbye Girl, The (1977) | Genre: Comedy, Romance | Rating: 4
- Sabrina (1954) | Genre: Comedy, Romance | Rating: 4
- My Fair Lady (1964) | Genre: Musical, Romance | Rating: 3
- African Queen, The (1951) | Genre: Action, Adventure, Romance, War | Rating: 3
- Rebecca (1940) | Genre: Romance, Thriller | Rating: 5
- Manhattan (1979) | Genre: Comedy, Drama, Romance | Rating: 4
- Roman Holiday (1953) | Genre: Comedy, Romance | Rating: 5
- They Might Be Gi

## UMUR SAMA, FILM SAMA, GENDER SAMA, BEDA OCCUPATION

PEKERJAAN 1

In [ ]:
proses_model()

Masukkan id user: 585
Masukkan id film 1: 162
Masukkan id film 2: 1289
Masukkan id film 3: 1856
 
User: user_585 | Sex: M | Age Group: 18-24 | Occupation: academic/educator
 
Film input:
- Crumb (1994) | Genre: Documentary | Rating: 4
- Koyaanisqatsi (1983) | Genre: Documentary, War | Rating: 3
- Kurt & Courtney (1998) | Genre: Documentary, Musical | Rating: 2
 
Film Rekomendasi:
- Roger & Me (1989) | Genre: Comedy, Documentary | Rating: 5
- Stop Making Sense (1984) | Genre: Documentary | Rating: 2
- Beetlejuice (1988) | Genre: Comedy, Fantasy | Rating: 4
- Princess Mononoke, The (Mononoke Hime) (1997) | Genre: Action, Adventure, Animation | Rating: 3
- Fast, Cheap & Out of Control (1997) | Genre: Documentary | Rating: 4
- Dogma (1999) | Genre: Comedy | Rating: 3
- City of Lost Children, The (1995) | Genre: Adventure, Sci-Fi | Rating: 4
- Star Wars: Episode I - The Phantom Menace (1999) | Genre: Action, Adventure, Fantasy, Sci-Fi | Rating: 4
- Akira (1988) | Genre: Adventure, Animation

In [ ]:
cek_history()

Masukkan id user: 585
 
User: user_585 | Sex: M | Age Group: 18-24 | Occupation: academic/educator
 
History Film User:
- Silence of the Lambs, The (1991) | Genre: Drama, Thriller | Rating: 4
- Back to the Future (1985) | Genre: Comedy, Sci-Fi | Rating: 4
- Fear and Loathing in Las Vegas (1998) | Genre: Comedy, Drama | Rating: 3
- One Flew Over the Cuckoo's Nest (1975) | Genre: Drama | Rating: 4
- Austin Powers: International Man of Mystery (1997) | Genre: Comedy | Rating: 4
- Bamboozled (2000) | Genre: Comedy | Rating: 4
- Best in Show (2000) | Genre: Comedy | Rating: 3
- Cell, The (2000) | Genre: Sci-Fi, Thriller | Rating: 5
- Godzilla 2000 (Gojira ni-sen mireniamu) (1999) | Genre: Action, Adventure, Sci-Fi | Rating: 5
- Ride with the Devil (1999) | Genre: Drama, Romance, War | Rating: 2
- Ghost in the Shell (Kokaku kidotai) (1995) | Genre: Animation, Sci-Fi | Rating: 4
- Rosencrantz and Guildenstern Are Dead (1990) | Genre: Comedy, Drama | Rating: 4
- Miller's Crossing (1990) | Genr

PEKERJAAN 2

In [ ]:
proses_model()

Masukkan id user: 777
Masukkan id film 1: 162
Masukkan id film 2: 1289
Masukkan id film 3: 1856
 
User: user_777 | Sex: M | Age Group: 18-24 | Occupation: unemployed
 
Film input:
- Crumb (1994) | Genre: Documentary | Rating: 4
- Koyaanisqatsi (1983) | Genre: Documentary, War | Rating: 3
- Kurt & Courtney (1998) | Genre: Documentary, Musical | Rating: 2
 
Film Rekomendasi:
- Dead Poets Society (1989) | Genre: Drama | Rating: 5
- Good Morning, Vietnam (1987) | Genre: Comedy, Drama, War | Rating: 3
- Rain Man (1988) | Genre: Drama | Rating: 3
- Glory (1989) | Genre: Action, Drama, War | Rating: 3
- Field of Dreams (1989) | Genre: Drama | Rating: 3
- Amadeus (1984) | Genre: Drama | Rating: 1
- Beetlejuice (1988) | Genre: Comedy, Fantasy | Rating: 4
- Die Hard (1988) | Genre: Action, Thriller | Rating: 4
- Raiders of the Lost Ark (1981) | Genre: Action, Adventure | Rating: 5
- Star Trek IV: The Voyage Home (1986) | Genre: Action, Adventure, Sci-Fi | Rating: 3
 
 
 


In [ ]:
cek_history()

Masukkan id user: 777
 
User: user_777 | Sex: M | Age Group: 18-24 | Occupation: unemployed
 
History Film User:
- Pee-wee's Big Adventure (1985) | Genre: Comedy | Rating: 2
- Breakfast Club, The (1985) | Genre: Comedy, Drama | Rating: 4
- Forrest Gump (1994) | Genre: Comedy, Romance, War | Rating: 3
- Fathers' Day (1997) | Genre: Comedy | Rating: 4
- White Men Can't Jump (1992) | Genre: Comedy | Rating: 4
- Alien (1979) | Genre: Action, Horror, Sci-Fi, Thriller | Rating: 4
- Back to the Future (1985) | Genre: Comedy, Sci-Fi | Rating: 4
- Aliens (1986) | Genre: Action, Sci-Fi, Thriller, War | Rating: 4
- Blade Runner (1982) | Genre: Film-Noir, Sci-Fi | Rating: 5
- Terminator 2: Judgment Day (1991) | Genre: Action, Sci-Fi, Thriller | Rating: 5
- Matrix, The (1999) | Genre: Action, Sci-Fi, Thriller | Rating: 4
- Star Wars: Episode IV - A New Hope (1977) | Genre: Action, Adventure, Fantasy, Sci-Fi | Rating: 3
- Star Wars: Episode V - The Empire Strikes Back (1980) | Genre: Action, Adventu

## UMUR BEDA, FILM SAMA, GENDER SAMA, OCCUPATION SAMA

In [ ]:
proses_model()

Masukkan id user: 737
Masukkan id film 1: 58
Masukkan id film 2: 140
Masukkan id film 3: 295
 
User: user_737 | Sex: M | Age Group: Under 18 | Occupation: unemployed
 
Film input:
- Postino, Il (The Postman) (1994) | Genre: Drama, Romance | Rating: 4
- Up Close and Personal (1996) | Genre: Drama, Romance | Rating: 3
- Pyromaniac's Love Story, A (1995) | Genre: Comedy, Romance | Rating: 3
 
Film Rekomendasi:
- Twister (1996) | Genre: Action, Adventure, Romance, Thriller | Rating: 3
- Mars Attacks! (1996) | Genre: Action, Comedy, Sci-Fi, War | Rating: 3
- Message in a Bottle (1999) | Genre: Romance | Rating: 5
- Runaway Bride (1999) | Genre: Comedy, Romance | Rating: 2
- Lost World: Jurassic Park, The (1997) | Genre: Action, Adventure, Sci-Fi, Thriller | Rating: 4
- What Dreams May Come (1998) | Genre: Drama, Romance | Rating: 4
- Evita (1996) | Genre: Drama, Musical | Rating: 2
- Robin Hood: Prince of Thieves (1991) | Genre: Drama | Rating: 5
- First Knight (1995) | Genre: Action, Adven

In [ ]:
cek_history()

Masukkan id user: 737
 
User: user_737 | Sex: M | Age Group: Under 18 | Occupation: unemployed
 
History Film User:
- Austin Powers: The Spy Who Shagged Me (1999) | Genre: Comedy | Rating: 2
- Godfather, The (1972) | Genre: Action, Crime, Drama | Rating: 4
- Batman & Robin (1997) | Genre: Action, Adventure, Crime | Rating: 3
- Big Sleep, The (1946) | Genre: Film-Noir, Mystery | Rating: 4
- Airplane! (1980) | Genre: Comedy | Rating: 4
- X-Men (2000) | Genre: Action, Sci-Fi | Rating: 5
- Men in Black (1997) | Genre: Action, Adventure, Comedy, Sci-Fi | Rating: 4
- Jurassic Park (1993) | Genre: Action, Adventure, Sci-Fi | Rating: 4
- Mission: Impossible (1996) | Genre: Action, Adventure, Mystery | Rating: 3
- Lost World: Jurassic Park, The (1997) | Genre: Action, Adventure, Sci-Fi, Thriller | Rating: 4
- Chinatown (1974) | Genre: Film-Noir, Mystery, Thriller | Rating: 5
- Shadow of a Doubt (1943) | Genre: Film-Noir, Thriller | Rating: 1
- Double Indemnity (1944) | Genre: Crime, Film-Noir |

In [ ]:
proses_model()

Masukkan id user: 2805
Masukkan id film 1: 58
Masukkan id film 2: 140
Masukkan id film 3: 295
 
User: user_2805 | Sex: M | Age Group: 56+ | Occupation: unemployed
 
Film input:
- Postino, Il (The Postman) (1994) | Genre: Drama, Romance | Rating: 4
- Up Close and Personal (1996) | Genre: Drama, Romance | Rating: 3
- Pyromaniac's Love Story, A (1995) | Genre: Comedy, Romance | Rating: 3
 
Film Rekomendasi:
- Grease (1978) | Genre: Comedy, Musical, Romance | Rating: 3
- Silence of the Lambs, The (1991) | Genre: Drama, Thriller | Rating: 4
- Dirty Dancing (1987) | Genre: Musical, Romance | Rating: 5
- Legends of the Fall (1994) | Genre: Drama, Romance, War, Western | Rating: 3
- Hot Shots! Part Deux (1993) | Genre: Action, Comedy, War | Rating: 4
- All That Jazz (1979) | Genre: Musical | Rating: 3
- South Pacific (1958) | Genre: Musical, Romance, War | Rating: 3
- Message in a Bottle (1999) | Genre: Romance | Rating: 5
- Godfather: Part III, The (1990) | Genre: Action, Crime, Drama | Ratin

In [ ]:
cek_history()

Masukkan id user: 2805
 
User: user_2805 | Sex: M | Age Group: 56+ | Occupation: unemployed
 
History Film User:
- Ferris Bueller's Day Off (1986) | Genre: Comedy | Rating: 3
- Star Wars: Episode IV - A New Hope (1977) | Genre: Action, Adventure, Fantasy, Sci-Fi | Rating: 3
- Fatal Attraction (1987) | Genre: Thriller | Rating: 3
- Sophie's Choice (1982) | Genre: Drama | Rating: 4
- Eyes of Tammy Faye, The (2000) | Genre: Documentary | Rating: 5
- Silence of the Lambs, The (1991) | Genre: Drama, Thriller | Rating: 4
- Tarzan and the Lost City (1998) | Genre: Action, Adventure | Rating: 3
- Once Were Warriors (1994) | Genre: Crime, Drama | Rating: 3
- Blue Collar (1978) | Genre: Crime, Drama | Rating: 4
- Suddenly, Last Summer (1959) | Genre: Drama | Rating: 5
- Godfather, The (1972) | Genre: Action, Crime, Drama | Rating: 4
- Godfather: Part II, The (1974) | Genre: Action, Crime, Drama | Rating: 4
- Psycho (1960) | Genre: Horror, Thriller | Rating: 1
- Lawrence of Arabia (1962) | Genre:

## UMUR SAMA, FILM SAMA, GENDER BEDA, OCCUPATION SAMA

In [ ]:
userid = "user_3075"
movie = ["movie_42", "movie_112", "movie_403"]
moviez = [movie_title_dict[ea_movie] for ea_movie in movie]
genres = [movie_genre[ea_movie] for ea_movie in movie]
rates = [movie_rating[ea_movie] for ea_movie in movie]
usersex = user_sex[userid]
userage = user_age[userid]
userocc = user_occ[userid]

recom = generate_recommendation(userid, movie, usersex, userage, userocc)
judul, genrez, ratingz = recom

print(" ")
print(f"User: {userid} | Sex: {usersex} | Age Group: {age_group_mapping.get(userage)} | Occupation: {occupation_mapping.get(userocc)}")
print(" ")

print("Film input:")
for i in range(3):
    hasil1 = [f"- {moviez[i]} | Genre: {genres[i]} | Rating: {rates[i]}"]
    for item in hasil1:
      print(item)

print(" ")

print("Film Rekomendasi:")
for i in range(11):
    try:
       hasil2 = [f"- {judul[i]} | Genre: {genrez[i]} | Rating: {ratingz[i]}"]
       for item in hasil2:
          print(item)
    except IndexError:
       print(" ")

print(" ")
print(" ")

 
User: user_3075 | Sex: F | Age Group: 25-34 | Occupation: lawyer
 
Film input:
- Dead Presidents (1995) | Genre: Action, Crime, Drama | Rating: 5
- Rumble in the Bronx (1995) | Genre: Action, Adventure, Crime | Rating: 3
- Two Crimes (1995) | Genre: Comedy, Crime, Drama | Rating: 4
 
Film Rekomendasi:
- American Beauty (1999) | Genre: Comedy, Drama | Rating: 2
- Godfather, The (1972) | Genre: Action, Crime, Drama | Rating: 4
- Manhattan (1979) | Genre: Comedy, Drama, Romance | Rating: 4
- Raiders of the Lost Ark (1981) | Genre: Action, Adventure | Rating: 5
- Escape from New York (1981) | Genre: Action, Adventure, Sci-Fi, Thriller | Rating: 5
- Purple Rose of Cairo, The (1985) | Genre: Comedy, Drama, Romance | Rating: 5
- Star Trek III: The Search for Spock (1984) | Genre: Action, Adventure, Sci-Fi | Rating: 4
- Hang 'em High (1967) | Genre: Western | Rating: 4
- Dances with Wolves (1990) | Genre: Adventure, Drama, Western | Rating: 3
- Englishman Who Went Up a Hill, But Came Down a 

In [ ]:
#cek histori 1
cek_history()

Masukkan id user: 3075
 
User: user_3075 | Sex: F | Age Group: 25-34 | Occupation: lawyer
 
History Film User:
- Schindler's List (1993) | Genre: Drama, War | Rating: 4
- L.A. Confidential (1997) | Genre: Crime, Film-Noir, Mystery, Thriller | Rating: 3
- One Night Stand (1997) | Genre: Drama | Rating: 4
- Witness (1985) | Genre: Drama, Romance, Thriller | Rating: 3
- Bank Dick, The (1940) | Genre: Comedy | Rating: 4
- Room with a View, A (1986) | Genre: Drama, Romance | Rating: 4
- Hocus Pocus (1993) | Genre: Children's, Comedy | Rating: 4
- Moonstruck (1987) | Genre: Comedy | Rating: 4
- Breaking the Waves (1996) | Genre: Drama | Rating: 5
- North by Northwest (1959) | Genre: Drama, Thriller | Rating: 5
- Annie Hall (1977) | Genre: Comedy, Romance | Rating: 3
- Fargo (1996) | Genre: Crime, Drama, Thriller | Rating: 5
- Manhattan (1979) | Genre: Comedy, Drama, Romance | Rating: 4
- Platoon (1986) | Genre: Drama, War | Rating: 4
- Jaws (1975) | Genre: Action, Horror | Rating: 3
- Sleepe

In [ ]:
userid = "user_836"
movie = ["movie_42", "movie_112", "movie_403"]
moviez = [movie_title_dict[ea_movie] for ea_movie in movie]
genres = [movie_genre[ea_movie] for ea_movie in movie]
rates = [movie_rating[ea_movie] for ea_movie in movie]
usersex = user_sex[userid]
userage = user_age[userid]
userocc = user_occ[userid]

recom = generate_recommendation(userid, movie, usersex, userage, userocc)
judul, genrez, ratingz = recom

print(" ")
print(f"User: {userid} | Sex: {usersex} | Age Group: {age_group_mapping.get(userage)} | Occupation: {occupation_mapping.get(userocc)}")
print(" ")

print("Film input:")
for i in range(3):
    hasil1 = [f"- {moviez[i]} | Genre: {genres[i]} | Rating: {rates[i]}"]
    for item in hasil1:
      print(item)

print(" ")

print("Film Rekomendasi:")
for i in range(11):
    try:
       hasil2 = [f"- {judul[i]} | Genre: {genrez[i]} | Rating: {ratingz[i]}"]
       for item in hasil2:
          print(item)
    except IndexError:
       print(" ")

print(" ")
print(" ")

 
User: user_836 | Sex: M | Age Group: 25-34 | Occupation: lawyer
 
Film input:
- Dead Presidents (1995) | Genre: Action, Crime, Drama | Rating: 5
- Rumble in the Bronx (1995) | Genre: Action, Adventure, Crime | Rating: 3
- Two Crimes (1995) | Genre: Comedy, Crime, Drama | Rating: 4
 
Film Rekomendasi:
- Lethal Weapon 2 (1989) | Genre: Action, Comedy, Crime, Drama | Rating: 3
- American Werewolf in London, An (1981) | Genre: Horror | Rating: 5
- Total Recall (1990) | Genre: Action, Adventure, Sci-Fi, Thriller | Rating: 4
- Predator (1987) | Genre: Action, Sci-Fi, Thriller | Rating: 4
- Highlander (1986) | Genre: Action, Adventure | Rating: 2
- Mission: Impossible (1996) | Genre: Action, Adventure, Mystery | Rating: 3
- Star Wars: Episode I - The Phantom Menace (1999) | Genre: Action, Adventure, Fantasy, Sci-Fi | Rating: 4
- Good Morning, Vietnam (1987) | Genre: Comedy, Drama, War | Rating: 3
- Men in Black (1997) | Genre: Action, Adventure, Comedy, Sci-Fi | Rating: 4
- Godfather, The (

In [ ]:
proses_model()

Masukkan id user: 836
Masukkan id film 1: 42
Masukkan id film 2: 112
Masukkan id film 3: 403
 
User: user_836 | Sex: M | Age Group: 25-34 | Occupation: lawyer
 
Film input:
- Dead Presidents (1995) | Genre: Action, Crime, Drama | Rating: 5
- Rumble in the Bronx (1995) | Genre: Action, Adventure, Crime | Rating: 3
- Two Crimes (1995) | Genre: Comedy, Crime, Drama | Rating: 4
 
Film Rekomendasi:
- Mission: Impossible (1996) | Genre: Action, Adventure, Mystery | Rating: 3
- Bedknobs and Broomsticks (1971) | Genre: Adventure, Children's, Musical | Rating: 3
- True Lies (1994) | Genre: Action, Adventure, Comedy, Romance | Rating: 2
- Hunt for Red October, The (1990) | Genre: Action, Thriller | Rating: 3
- Jurassic Park (1993) | Genre: Action, Adventure, Sci-Fi | Rating: 4
- Romancing the Stone (1984) | Genre: Action, Adventure, Comedy, Romance | Rating: 3
- Mask of Zorro, The (1998) | Genre: Action, Adventure, Romance | Rating: 4
- Speed (1994) | Genre: Action, Romance, Thriller | Rating: 4

In [ ]:
cek_history()

Masukkan id user: 836
 
User: user_836 | Sex: M | Age Group: 25-34 | Occupation: lawyer
 
History Film User:
- Oliver & Company (1988) | Genre: Animation, Children's | Rating: 4
- Home Alone (1990) | Genre: Children's, Comedy | Rating: 3
- Alice in Wonderland (1951) | Genre: Animation, Children's, Musical | Rating: 4
- Beavis and Butt-head Do America (1996) | Genre: Animation, Comedy | Rating: 3
- Big (1988) | Genre: Comedy, Fantasy | Rating: 2
- Star Kid (1997) | Genre: Adventure, Children's, Fantasy, Sci-Fi | Rating: 3
- Star Wars: Episode V - The Empire Strikes Back (1980) | Genre: Action, Adventure, Drama, Sci-Fi, War | Rating: 4
- Mystery Men (1999) | Genre: Action, Adventure, Comedy | Rating: 4
- Shanghai Noon (2000) | Genre: Action | Rating: 3
- Ready to Rumble (2000) | Genre: Comedy | Rating: 2
- Erin Brockovich (2000) | Genre: Drama | Rating: 3
- Nurse Betty (2000) | Genre: Comedy, Thriller | Rating: 5
- Keeping the Faith (2000) | Genre: Comedy, Romance | Rating: 4
- Best in S